# K-Means para scoring crediticio (CRISP-DM)
Segmentación de clientes y análisis de mora/PD sin leakage.

Ajusta las variables de configuración y ejecuta las celdas en orden.

## DependenciasImporta librerías necesarias para preprocesamiento, clustering (K-Means), evaluación (`silhouette`) y visualización (PCA y gráficos).

In [ ]:
import numpy as npimport pandas as pdfrom pathlib import Pathfrom sklearn.preprocessing import StandardScalerfrom sklearn.cluster import KMeansfrom sklearn.metrics import silhouette_scorefrom sklearn.decomposition import PCAimport matplotlib.pyplot as pltimport seaborn as sns

## Configuración del experimentoDefine rutas, columnas objetivo e identificador, cantidad de clusters y semilla. Usa rutas relativas del repo para compatibilidad en GitHub.

In [ ]:
TRAIN_CSV = 'data/train.csv'TARGET_COLUMN = 'mora'ID_COLUMN = 'customer_id'FEATURE_COLUMNS = NoneN_CLUSTERS = 5RANDOM_STATE = 42OUT_DIR = 'outputs/unsupervised/kmeans_nb'PREDICTIONS_PATH = None

## Carga de datos y selección de variablesLee el CSV de entrenamiento. Incluye un *fallback* a `../data/train.csv` si ejecutas el notebook desde `unsupervised/`. Excluye `mora` y `customer_id` de las features para evitar leakage y sesgos.

In [ ]:
from pathlib import Pathimport pandas as pdout_dir = Path(OUT_DIR)out_dir.mkdir(parents=True, exist_ok=True)csv_path = Path(TRAIN_CSV)if not csv_path.exists():
    alt = Path('..') / TRAIN_CSV
    if alt.exists():
        csv_path = altprint(f'Usando TRAIN_CSV: {csv_path.resolve()}')import osif not csv_path.exists():
    raise FileNotFoundError(f'No se encuentra {csv_path}. Usa "data/train.csv" si ejecutas en la raíz del repo, o "../data/train.csv" si ejecutas desde la carpeta del notebook.')df = pd.read_csv(csv_path)X = df[FEATURE_COLUMNS] if FEATURE_COLUMNS else df.select_dtypes(include=[np.number])if TARGET_COLUMN and TARGET_COLUMN in X.columns:
    X = X.drop(columns=[TARGET_COLUMN])if ID_COLUMN and ID_COLUMN in X.columns:
    X = X.drop(columns=[ID_COLUMN])

## PreprocesamientoTratamiento de `NaN` y escalado estandar (`StandardScaler`) para homogeneizar las escalas antes de agrupar.

In [ ]:
X = X.replace([np.inf, -np.inf], np.nan)X = X.fillna(X.median(numeric_only=True))scaler = StandardScaler()X_scaled = scaler.fit_transform(X.values)

## Búsqueda de k por `silhouette`Evalúa `k` en el rango 2–8 y selecciona el que maximiza `silhouette`. Guarda `k_search.csv` con los resultados.

In [ ]:
ks = list(range(2, 9))results = []for k in ks:
    m = KMeans(n_clusters=k, n_init=10, random_state=RANDOM_STATE)
    lab = m.fit_predict(X_scaled)
    s = float(silhouette_score(X_scaled, lab)) if len(set(lab)) > 1 else float('nan')
    results.append({'k': k, 'silhouette': s})res_df = pd.DataFrame(results)best_k = int(res_df.sort_values('silhouette', ascending=False).iloc[0]['k'])N_CLUSTERS = best_kres_df.to_csv(out_dir / 'k_search.csv', index=False)best_k

## Curva `silhouette` vs `k`Visualiza la calidad del agrupamiento para cada `k` y guarda `silhouette_vs_k.png`.

In [ ]:
plt.figure(figsize=(6,4))plt.plot(res_df['k'], res_df['silhouette'], marker='o')plt.xlabel('k')plt.ylabel('silhouette')plt.title('Silhouette vs k')plt.grid(True, alpha=0.3)plt.tight_layout()plt.savefig(out_dir / 'silhouette_vs_k.png', dpi=150)plt.show()

## Entrenamiento final y métricaEjecuta K-Means con el `k` óptimo y calcula `silhouette` sobre las asignaciones finales.

## Conclusiones y aplicabilidad
- Los clusters presentan tasas de mora diferenciadas, útiles para segmentación de riesgo.
- El índice de cluster puede incorporarse como feature en el modelo supervisado y validarse.
- La visualización PCA ayuda a interpretar la separación; si hay solapamiento, ajustar `k` y variables.
- Evitar leakage: `mora` no se emplea en el clustering; solo en análisis por cluster.

In [ ]:
model = KMeans(n_clusters=N_CLUSTERS, n_init=10, random_state=RANDOM_STATE)labels = model.fit_predict(X_scaled)sil = float(silhouette_score(X_scaled, labels)) if len(set(labels)) > 1 else float('nan')sil

## Resumen por clusterCalcula tamaño y tasa de mora por cluster. Si proporcionas PD del modelo supervisado (para el mismo entrenamiento), promedia por cluster para complementar el análisis.

In [ ]:
df_out = df.copy()df_out['cluster'] = labelssummaries = []groups = df_out.groupby('cluster')for cid, g in groups:
    size = int(len(g))
    default_rate = None
    if TARGET_COLUMN and TARGET_COLUMN in g.columns:
        default_rate = float(g[TARGET_COLUMN].mean()) if g[TARGET_COLUMN].dropna().nunique() <= 2 else float((g[TARGET_COLUMN] > 0).mean())
    mean_pd = None
    if PREDICTIONS_PATH and ID_COLUMN:
        preds = pd.read_csv(PREDICTIONS_PATH)
        if 'pred_proba' in preds.columns and ID_COLUMN in preds.columns and ID_COLUMN in g.columns:
            merged = g[[ID_COLUMN]].merge(preds[[ID_COLUMN, 'pred_proba']], on=ID_COLUMN, how='left')
            mean_pd = float(merged['pred_proba'].mean())
    summaries.append({'cluster': int(cid), 'size': size, 'default_rate': default_rate, 'mean_pred_pd': mean_pd})summary_df = pd.DataFrame(summaries).sort_values('cluster')summary_df

## Visualización PCAProyección a 2 componentes principales para inspección visual de la separación entre clusters y relación con mora. Guarda `clusters_pca.png` y `default_pca.png`.

In [ ]:
pca = PCA(n_components=2, random_state=42)comps = pca.fit_transform(X_scaled)dfp = pd.DataFrame({'pc1': comps[:,0], 'pc2': comps[:,1], 'cluster': labels})plt.figure(figsize=(8,6))sns.scatterplot(data=dfp, x='pc1', y='pc2', hue='cluster', palette='tab10', s=12, linewidth=0)plt.tight_layout()plt.savefig(out_dir / 'clusters_pca.png', dpi=150)plt.show()if TARGET_COLUMN and TARGET_COLUMN in df.columns:
    dfp2 = dfp.copy()
    dfp2['default'] = (df[TARGET_COLUMN].values > 0).astype(int)
    plt.figure(figsize=(8,6))
    sns.scatterplot(data=dfp2, x='pc1', y='pc2', hue='default', palette='Set1', s=12, linewidth=0)
    plt.tight_layout()
    plt.savefig(out_dir / 'default_pca.png', dpi=150)
    plt.show()

## Guardado de resultadosPersistencia de configuración, métricas, asignaciones y resumen de clusters para reproducibilidad y análisis posterior.

In [ ]:
import jsonconfig = {'train_csv': TRAIN_CSV, 'target_column': TARGET_COLUMN, 'id_column': ID_COLUMN, 'feature_columns': FEATURE_COLUMNS if FEATURE_COLUMNS else list(X.columns), 'n_clusters': N_CLUSTERS, 'random_state': RANDOM_STATE, 'predictions_path': PREDICTIONS_PATH}with open(out_dir / 'config.json', 'w', encoding='utf-8') as f:
    json.dump(config, f, ensure_ascii=False, indent=2)metrics = {'silhouette': sil}with open(out_dir / 'metrics.json', 'w', encoding='utf-8') as f:
    json.dump(metrics, f, ensure_ascii=False, indent=2)assign_cols = [ID_COLUMN, 'cluster'] if ID_COLUMN else ['cluster']df_out[assign_cols].to_csv(out_dir / 'cluster_assignments.csv', index=False)summary_df.to_csv(out_dir / 'cluster_summary.csv', index=False)

Ajusta `TRAIN_CSV`, `TARGET_COLUMN`, `ID_COLUMN`, `N_CLUSTERS` y ejecuta secuencialmente.